In [1]:
from collections import defaultdict
from tqdm import tqdm



def get_aliases(fp):
    dic = defaultdict(list)
    with open(fp, "r", encoding="utf-8") as f:
        l_ns = sum(1 for _ in f)
        f.seek(0)
        for line in tqdm(f, total=l_ns, desc="reading aliases"):
            try:
                split_line = line.strip().split("\t")
                entity_id = str(split_line[0])
                entity_name = str(split_line[1])
                aliases = split_line[2:]
                dic[entity_id].append(entity_name)
                for al in aliases:
                    dic[entity_id].append(str(al))

            except ValueError as e:
                print(f"The line has not enough arguments: {line.strip()}")
                break
    return dic

In [2]:

root = "."
RAW_FOLDER = f"{root}/data/raw"

RAW_TXT_FILES ={
    "descriptions": f"{RAW_FOLDER}/wikidata5m_text.txt",
    "aliases": f"{RAW_FOLDER}/wikidata5m_entity.txt",
    "relations": f"{RAW_FOLDER}/wikidata5m_relation.txt",
    "triples": f"{RAW_FOLDER}/wikidata5m_transductive_train.txt",
}
aliases_raw_fp = RAW_TXT_FILES["aliases"]

aliases_dict_all = get_aliases(aliases_raw_fp)


reading aliases: 100%|██████████| 4813491/4813491 [00:22<00:00, 213939.33it/s]


In [4]:
aliases_dict_all
